In [2]:
import torch
import signatory
import numpy as np
import itertools
import trading_strategy
import utils

depth = 2
delta = 0.3
ts = trading_strategy.TradingStrategy(depth, delta)
batch_size = 2
d = 3 # number of stocks
N = 1 # size of info
Z_dim = N+d+1 # dimension of Z
T = 7 # number of time steps
batch_stock_data = 1+torch.rand(batch_size, T, d) # X
batch_info_data = 1+torch.rand(batch_size, T, N) # f
ts.fit(batch_stock_data, batch_info_data)

function compute_lead_lag_transform took 0.00s
function compute_signature took 0.07s
mu_sig successfully computed
function compute_mu_sig took 0.00s
sigma_sig successfully computed
function compute_sigma_sig took 1.65s
Lambda successfully computed
computing self.lambda_ = 2* sqrt(13612384.0/0.3)
function compute_lambda took 0.18s
Fitting successful


In [3]:
ts.functionals

[tensor([ 1.1517e+01,  4.0008e+00, -2.0482e+00,  1.6901e+01,  5.8298e+01,
          1.8910e+01, -6.2926e-02, -1.3609e+00,  1.1506e+01, -1.0887e+01,
          1.4500e+01,  1.6797e+00,  2.7859e+01, -1.8747e+01, -3.9969e+01,
          9.9961e+00,  8.0641e+00, -4.1340e+00,  2.9122e+01, -2.8309e+00,
          4.0479e+01, -6.1382e+00, -1.0586e+02,  3.7326e-01,  1.2859e+01,
          3.6329e+01,  5.8588e+00, -2.1448e+01,  3.8150e+00, -2.6012e+01,
         -8.9168e+00]),
 tensor([ -33.4089,  -11.4080,   -6.2252,   10.9742,  186.7463,  102.4778,
           23.1741,   44.5590,    0.9721,   78.1981,   73.0726,   39.2729,
           45.5602,  136.0712,   -9.0333, -255.3709, -103.6024,   -8.2259,
          225.7846,   54.3803,   77.9545,  121.1893,   78.3454, -180.2494,
          196.8532, -149.5856,  -70.2695,   12.5203,   55.4418,  153.1961,
          -40.9346]),
 tensor([-127.2383,  -56.7573,  197.4228,   19.5160,  104.7580,   58.6915,
           -6.7284,   18.4334,   20.3924,   11.1339,   -5.54